# Using Open-source HuggingFace Models to Generate QAs from Raw Data in JSON format

In this example, we will show you how to generate question-answers (QAs) from given text strings using open-source Huggingface models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install libraries

In [2]:
!{sys.executable} -m pip install -q transformers accelerate bitsandbytes scipy

### Import dependency

In [3]:
from IPython.display import display

from uniflow.flow.client import TransformClient
from uniflow.flow.config import HuggingfaceModelConfig, TransformQAHuggingFaceJsonFormatConfig
from uniflow.op.prompt import Context
from pprint import pprint
import time

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `PromptTemplate` class. However, since we are using the default `PromptTemplate` in this example, we will not create it separately.

Second, we craft some dummy sample raw text strings. Below, we build a dataset with some text strings.

In [4]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

raw_context_input = raw_context_input * 250

Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [5]:

input_data = [
    Context(context=data)
    for data in raw_context_input
]

print("sample size of processed input data: ", len(input_data))

print("\nExample uniflow context data:")
pprint(input_data[:2])


sample size of processed input data:  1000

Example uniflow context data:
[Context(context='We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.'),
 Context(context='In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first 

### Use LLM to generate data

In this example, we will use the [TransformQAHuggingFaceJsonFormatConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/config.py#L128)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here in this example, we use our default `PromptTemplate` to the `TransformQAHuggingFaceJsonFormatConfig`, but you can use your customized instructions and examples instead if you want.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [6]:
config = TransformQAHuggingFaceJsonFormatConfig(
    model_config=HuggingfaceModelConfig(
            batch_size=64,
            model_name="google/gemma-7b-it"
        )
)
client = TransformClient(config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
model-00002-of-00003.safetensors:   0%|                                                                            | 0.00/5.00G [00:00<?, ?B/s]
model-00002-of-00003.safetensors:   0%|▎                                                                   | 21.0M/5.00G [00:00<00:28, 174MB/s]
model-00002-of-00003.safetensors:   1%|▋                                                                   | 52.4M/5.00G [00:00<00:23, 215MB/s]
model-00002-of-00003.safetensors:   2%|█▏                                                                  | 83.9M/5.00G [00:00<00:21, 227MB/s]
model-00002-of-00003.safetensors:   2%|█▌                                                                   | 115M/5.00G [00:00<00:21, 230MB/s]
model-00002-of-00003.safetensors:   3%|██                                              

ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
print("HuggingFace model config: ")
pprint(config)

prompt = config.prompt_template.instruction
print("\nPrompt: ", prompt)

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [ ]:
start_time = time.time()
outputs = client.run(input_data)
execution_time = time.time() - start_time

In [ ]:
print("execution time: ", execution_time, " seconds")

### Process the output

Let's take a look of the generated output, which is already a list of JSON

In [ ]:
print("Sample output:")
pprint(outputs[0]["output"][0]["response"][0], sort_dicts=False)

### Count number of tokens being processed per second

In [ ]:

def count_tokens(document):
    ''' 
    Give a document(string), return total number of tokens(words)

    Parameters:
       documents(string): A document

    Returns:
       count(int): Total number of tokens. 1 token = 0.75 word
    '''
    num_words = len(document.split())
    num_tokens = round(num_words/0.75)
    return num_tokens

In [ ]:
# Count tokens
input_tokens = sum(count_tokens(d) for d in raw_context_input)
prompt_tokens = count_tokens(config.prompt_template.instruction)
output_tokens = 0
for output in outputs:
    if output["output"]:
        for response in output["output"]:
            for r in response["response"]:
                output_tokens += count_tokens(r["context"])
                output_tokens += count_tokens(r["question"])
                output_tokens += count_tokens(r["answer"])

tokens = input_tokens + prompt_tokens + output_tokens
print("output tokens: ", output_tokens)
print("input tokens: ", input_tokens)
print("prompt tokens: ",prompt_tokens)
print("total tokens: ", tokens)
print("total tokens processed per sec: ", round(tokens/execution_time))

Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `TransformQAHuggingFaceJsonFormatConfig` flow. The answer is "It depends on your data token length, your GPU memory, your LLM size, etc." In the following experiment, we use an AWS `g5.xlarge` instance that has a GPU with 24G memory and a quantized LLM (2G). We still use the above raw data strings `raw_context_input`.


Here are the results:
      
- batch_size = 8     
    100%|██████████| 125/125 [1:06:57<00:00, 32.14s/it]            
    output tokens:  115000   
    input tokens:  76000   
    prompt tokens:  43   
    total tokens:  191043   
    total tokens processed per sec:  48     
      
- batch_size = 64    
    100%|██████████| 16/16 [10:17<00:00, 38.62s/it]        
    output tokens:  107840    
    input tokens:  76000    
    prompt tokens:  43    
    total tokens:  183883    
    total tokens processed per sec:  298     
 

## End of the notebook

Check more Uniflow use cases in the [example folder](https://github.com/CambioML/uniflow/tree/main/example/model#examples)!

<a href="https://www.cambioml.com/" title="Title">
    <img src="../image/cambioml_logo_large.png" style="height: 100px; display: block; margin-left: auto; margin-right: auto;"/>
</a>